<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Анализ-данных-и-предобработка" data-toc-modified-id="Анализ-данных-и-предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Анализ данных и предобработка</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-DummyRegressor" data-toc-modified-id="Модель-DummyRegressor-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель DummyRegressor</a></span></li><li><span><a href="#Модель-LinearRegression" data-toc-modified-id="Модель-LinearRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LinearRegression</a></span></li><li><span><a href="#Модель--CatBoostRegressor-на-данных-с-OHE" data-toc-modified-id="Модель--CatBoostRegressor-на-данных-с-OHE-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель  CatBoostRegressor на данных с OHE</a></span></li><li><span><a href="#Модель--CatBoostRegressor-на-данных-без-OHE" data-toc-modified-id="Модель--CatBoostRegressor-на-данных-без-OHE-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель  CatBoostRegressor на данных без OHE</a></span></li><li><span><a href="#Модель-LightGBMRegressor" data-toc-modified-id="Модель-LightGBMRegressor-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Модель LightGBMRegressor</a></span></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import timeit

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV


from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

from catboost import CatBoostRegressor 
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

## Подготовка данных

In [10]:
try:
    data = pd.read_csv('/Users/mikhail/Desktop/praktikum/project_11/autos.csv')
    
except:
    data = pd.read_csv('/datasets/autos.csv')

### Знакомство с данными

In [3]:
# ознакомимся с данными

data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
# посмотрим количество строк, тип данных в столбцах и пропуски

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
# посмотрим на числовые признаки

data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### Анализ данных и предобработка

Для начала разберёмся с пропусками в наших данных

In [6]:
#посчитаем пропуски и их процент в каждом столбце 

report = data.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of missing values'] = ((report['missing_values'] / data.shape[0])*100).round(2)
report.drop(columns = 'missing_values')
report_missing = report.sort_values(by = '% of missing values', ascending = False)
report_missing

,missing_values,% of missing values
NotRepaired,71154,20.08
VehicleType,37490,10.58
FuelType,32895,9.28
Gearbox,19833,5.60
Model,19705,5.56
DateCrawled,0,0.00
Price,0,0.00
RegistrationYear,0,0.00
Power,0,0.00
Kilometer,0,0.00


Более всего пропусков имеет столбец 'NotRepaired' - 20%. Остальные столбцы с пропусками ('VehicleType', 'FuelType', 'Gearbox', 'Model') имеют в 2 и больше раза менньше попусков, но несут для нас важнейшую информацию - хотелось бы их сохранить. 

In [7]:
missing_columns = data[['NotRepaired', 'VehicleType', 'FuelType', 'Gearbox', 'Model']]

In [8]:
for i in missing_columns:
    display(data[i].name)
    display(data[i].value_counts())

'NotRepaired'

no     247161
yes     36054
Name: NotRepaired, dtype: int64

'VehicleType'

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

'FuelType'

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

'Gearbox'

manual    268251
auto       66285
Name: Gearbox, dtype: int64

'Model'

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64

В столбце 'NotRepaired' довольно много пропусков, чтобы пожертвовать таким большим объемом данных. Поэтому предлагаю ввести еще одну категорию для для данного признака 'no data' и заменить пропуски. 

In [9]:
data['NotRepaired'].fillna('no data', inplace = True)

Пропусков в столбцe 'VehicleType' порядка 7% от общего числа строк можно было бы избавиться, но лучше поступить, как с с пропусками в 'NotRepaired' и заменить их на 'no data'

In [10]:
data['VehicleType'].fillna('no data', inplace = True)

Признак 'FuelType' имеет две наиболее популярные категории: petrol и gasoline. Которые при этом обозначают одно и то же - бензин. А значит мы имеем явный перекос в сторону одного типа топлива, к которму и сможем приписать наши пропуски.

In [11]:
# приводим gasoline и petrol к одному значению, пусть это будет petrol 

data['FuelType'] = data['FuelType'].replace({'gasoline' : 'petrol'})

# заполняем пропуски значение petrol

data['FuelType'].fillna('petrol', inplace = True)

Столбец 'Gearbox' и его 5,5% пропусков - здесь две категории и приписать 20 тысяч пропусков к какой-либо из них явно подпортит нашу выборку. Поэтому создадим новую категорию.

In [12]:
data['Gearbox'].fillna('no data', inplace = True)

В признаке 'Model', весьма важном, предпочту от пропусков избавиться.

In [13]:
data.dropna(subset = ['Model'], inplace = True)

In [14]:
# проверяем насколько удачно прошла процедура избавления от пропусков

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334664 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        334664 non-null  object
 1   Price              334664 non-null  int64 
 2   VehicleType        334664 non-null  object
 3   RegistrationYear   334664 non-null  int64 
 4   Gearbox            334664 non-null  object
 5   Power              334664 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          334664 non-null  int64 
 8   RegistrationMonth  334664 non-null  int64 
 9   FuelType           334664 non-null  object
 10  Brand              334664 non-null  object
 11  NotRepaired        334664 non-null  object
 12  DateCreated        334664 non-null  object
 13  NumberOfPictures   334664 non-null  int64 
 14  PostalCode         334664 non-null  int64 
 15  LastSeen           334664 non-null  object
dtypes: int64(7), object(

Пройдемся по остальным столбцам и проверим данные на ошибки

In [15]:
# изучим подробнее столбец 'Price'
data['Price'].describe()

count    334664.000000
mean       4504.346790
std        4531.438572
min           0.000000
25%        1150.000000
50%        2800.000000
75%        6500.000000
max       20000.000000
Name: Price, dtype: float64

In [16]:
# мы имеем значения '0' в поле с ценой, а значит стоит посмотреть сколько таковых

len(data[data['Price'] == 0])

8588

В данном случае у нас 8588 объектов с ценой 0 и заполнить мы их никак не сможем. С учетом того, что данный столбец это наш целевой признак - придется избавиться от этих строк.

In [17]:
data = data.query('Price != 0')

Проверим столбец с годом регистрации авто

In [18]:
data['RegistrationYear'].value_counts()

2000    21301
1999    20959
2005    20561
2006    19150
2003    18878
        ...  
1941        1
4500        1
2200        1
1949        1
2290        1
Name: RegistrationYear, Length: 122, dtype: int64

В данном столбце наблюдаются некоторые аномалии с годом - присутствуют такие значения, как: 4100, 1200, 5300, 8888, 2290 и тд. Я предпочитаю избавиться от данных выбросов, определив адекватный временной промежуток с 1950 по 2021. Тем самым данные будут избавлены от выбросов.

In [19]:
data = data.query('1950 <= RegistrationYear <= 2021')

In [20]:
# столбец 'Power' имеет минимальные значения, равные 0
# посчитаем количество нулевых значений

len(data[data['Power'] == 0])

31235

Более 40 тысяч строк, где мощность равна 0. Попробуем заменить эти 0 на медианные значения мощности по моделям.

In [21]:
# группируем по моделям и считаем медианные значения мощности
model_power = data.query('Power != 0').groupby('Model')['Power'].median()

# создаем словарь для заполнения
model_power_dict = model_power.to_dict()

In [22]:
# меняем 0 на Nan для удобства заполнения
data['Power'].replace(0, np.nan, inplace=True)

# заполняем пропуски из словаря
data['Power'] = data['Power'].fillna(data['Model'].map(model_power_dict))

# на всякий случай дропаем пропуски 
data.dropna(subset = ['Power'], inplace = True)

# перводим значения в целочисленные
data['Power'] = data['Power'].astype(int)


In [23]:
#  имеются нулевые значения в столбце с месяцем публикации

len(data[data['RegistrationMonth'] == 0])

27532

В целом это не критичный признак, поэтому можно оставить все без изменений.

In [24]:
# проверяем и заодно знакомимся с данными столбца 'Brand'

data['Brand'].value_counts()

volkswagen       71263
opel             36816
bmw              34065
mercedes_benz    30254
audi             27465
ford             23519
renault          16693
peugeot           9949
fiat              8864
seat              6502
skoda             5307
mazda             5151
smart             4928
citroen           4793
nissan            4618
toyota            4399
hyundai           3313
mini              3108
volvo             3026
mitsubishi        2782
honda             2581
kia               2298
suzuki            2146
alfa_romeo        2110
chevrolet         1606
chrysler          1285
dacia              871
daihatsu           739
porsche            697
subaru             696
jeep               630
land_rover         519
saab               502
daewoo             490
jaguar             483
lancia             434
trabant            425
rover              390
lada               201
Name: Brand, dtype: int64

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325918 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        325918 non-null  object
 1   Price              325918 non-null  int64 
 2   VehicleType        325918 non-null  object
 3   RegistrationYear   325918 non-null  int64 
 4   Gearbox            325918 non-null  object
 5   Power              325918 non-null  int64 
 6   Model              325918 non-null  object
 7   Kilometer          325918 non-null  int64 
 8   RegistrationMonth  325918 non-null  int64 
 9   FuelType           325918 non-null  object
 10  Brand              325918 non-null  object
 11  NotRepaired        325918 non-null  object
 12  DateCreated        325918 non-null  object
 13  NumberOfPictures   325918 non-null  int64 
 14  PostalCode         325918 non-null  int64 
 15  LastSeen           325918 non-null  object
dtypes: int64(7), object(

Избавимся от столбцов, не несущих никакой важной информации для наших моделей:

In [26]:
data = data.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325918 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              325918 non-null  int64 
 1   VehicleType        325918 non-null  object
 2   RegistrationYear   325918 non-null  int64 
 3   Gearbox            325918 non-null  object
 4   Power              325918 non-null  int64 
 5   Model              325918 non-null  object
 6   Kilometer          325918 non-null  int64 
 7   RegistrationMonth  325918 non-null  int64 
 8   FuelType           325918 non-null  object
 9   Brand              325918 non-null  object
 10  NotRepaired        325918 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.8+ MB


В данном случае мы имеем дело с пропусками исключительно в категориальных признаках. 

### Вывод

В нашем датафрейме было 5 столбцов с содержанием пропусков от 5 до 20% данных. 

Вот эти столбцы:
- NotRepaired - 71154 пропусков -	20.08% данных
- VehicleType	- 37490 пропусков - 10.58% данных
- FuelType - 32895 пропусков - 9.28% данных
- Gearbox - 19833 пропусков - 5.60% данных
- Model - 19705 пропусков - 5.56% данных

Так же были обнаружены объекты с некорректными или с незаполненными данными (цена, мощность и год регистрации). 

В целом - пришлось избавиться порядка от 10% объектов, что, на мой взгляд, не так критично.

Так же были удалены столбцы, не имеющие никакой важности при обучении моделей.

## Обучение моделей

In [28]:
# сохраянем фичи и таргет для прогона модели по данным без OHE

X_no_ohe = data[['RegistrationYear','Power','Kilometer','RegistrationMonth', \
                 'VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']]
y_no_ohe = data['Price']

In [29]:
# проводим OHE для категориальных признаков

X_ohe = pd.get_dummies(data[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']], drop_first = True)
data[X_ohe.columns] = X_ohe
X_columns = ['RegistrationYear','Power','Kilometer','RegistrationMonth'] + list(X_ohe.columns)

In [30]:
# делим данные на выборки

X = data[X_columns]
y = data['Price']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=1234)
X_train_no_ohe, X_valid_no_ohe, y_train_no_ohe, y_valid_no_ohe = \
train_test_split(X_no_ohe, y_no_ohe, test_size=0.25, random_state=12345)

In [31]:
# запишем функцию для обучения моделей и нахождения значения RMSE

def score(model, target = y_train, features = X_train):
    start_time = timeit.default_timer()
    score = mean_squared_error(target, model.predict(features))**(1/2)
    predict_time = timeit.default_timer() - start_time
    return score, predict_time
def fit(model):
    start_time = timeit.default_timer()
    model.fit(X_train, y_train)
    fit_time = timeit.default_timer() - start_time
    return fit_time

### Модель DummyRegressor

In [32]:
dumr = DummyRegressor(strategy="median")
dumr_fit_time = fit(dumr)
print('Время обучения модели:', dumr_fit_time)
Score, time  = score(dumr)
print('Значение RMSE:', Score)
print('Время предсказания:', time)

Время обучения модели: 0.006542541999998264
Значение RMSE: 4831.102242128869
Время предсказания: 0.007224994999997847


### Модель LinearRegression

In [33]:
lr = LinearRegression()
lr_fit_time = fit(lr)
print('Время обучения модели:', lr_fit_time)
Score, time  = score(lr)
print('Значение RMSE:', Score)
print('Время предсказания:', time)

Время обучения модели: 22.980423431
Значение RMSE: 2772.2337512833424
Время предсказания: 3.664582834000001


### Модель  CatBoostRegressor на данных с OHE

In [34]:
#%%time
cbr = CatBoostRegressor(iterations = 100, learning_rate = 0.6, depth = 10)
start_time = timeit.default_timer()
cbr.fit(X_train, y_train, silent= True)
cbr_fit_time = timeit.default_timer() - start_time
print('Время обучения модели:', cbr_fit_time)
Score, time  = score(cbr)
print('Значение RMSE:', Score)
print('Время предсказания:', time)

Время обучения модели: 19.242516989000002
Значение RMSE: 1507.3402408816353
Время предсказания: 0.5976547650000015


### Модель  CatBoostRegressor на данных без OHE

In [35]:
cbr_no_ohe = CatBoostRegressor(iterations = 100, learning_rate = 0.6, depth = 10)
start_time = timeit.default_timer()
cbr_no_ohe.fit(X_train_no_ohe, y_train_no_ohe, silent= True, cat_features = \
               ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired'])
cbr_no_ohe_fit_time = timeit.default_timer() - start_time
print('Время обучения модели:', cbr_no_ohe_fit_time)
Score, time  = score(cbr_no_ohe, y_train_no_ohe, X_train_no_ohe)
print('Значение RMSE:', Score)
print('Время предсказания:', time)

Время обучения модели: 44.74082296899999
Значение RMSE: 1462.8031639521241
Время предсказания: 1.8001342350000016


### Модель LightGBMRegressor

In [36]:
lgmbr = LGBMRegressor(max_depth = 200, num_leaves = 240, n_estimators = 340)
start_time = timeit.default_timer()
lgmbr_fit_time = fit(lgmbr)
print('Время обучения модели:', lgmbr_fit_time)
Score, time  = score(lgmbr)
print('Значение RMSE:', Score)
print('Время предсказания:', time)

Время обучения модели: 23.082368173000006
Значение RMSE: 1270.0828559971912
Время предсказания: 19.48762870600001


## Анализ моделей

Проверим все наши модели на валидационной выборке

In [37]:
dumr_score, dumr_time  = score(dumr, target = y_valid, features = X_valid)
print('Значение RMSE:', dumr_score)
print('Время предсказания:', dumr_time)

Значение RMSE: 4822.094620406588
Время предсказания: 0.003933953999990081


In [38]:
lr_score, lr_time  = score(lr, target = y_valid, features = X_valid)
print('Значение RMSE:', lr_score)
print('Время предсказания:', lr_time)

Значение RMSE: 2782.322241321886
Время предсказания: 1.3096212019999882


In [39]:
cbr_score, cbr_time  = score(cbr, target = y_valid, features = X_valid)
print('RMSE score:', cbr_score)
print('время предсказания:', cbr_time)

RMSE score: 1614.579968913522
время предсказания: 0.18305274700000496


In [40]:
cbr_no_ohe_score, cbr_no_ohe_time  = score(cbr_no_ohe, target = y_valid_no_ohe, features = X_valid_no_ohe)
print('RMSE score:', cbr_no_ohe_score)
print('время предсказания:', cbr_no_ohe_time)

RMSE score: 1640.6096458973527
время предсказания: 0.5045836980000047


In [41]:
lgmbr_score, lgmbr_time  = score(lgmbr, target = y_valid, features = X_valid)
print('RMSE score:', lgmbr_score)
print('время предсказания:', lgmbr_time)

RMSE score: 1535.820784692363
время предсказания: 6.116650214000003


Соберем все данные по моделям в одном фрейме

In [47]:
columns = ['Значение RMSE', 'Время предсказания', 'Время обучения', 'Общее время']
index = ['DummyRegressor', 'LinearRegression','CatBoostRegressor with OHE', \
         'CatBoostRegressor without OHE', 'LightGBMRegressor']
report = [[dumr_score, dumr_time, dumr_time, dumr_time + dumr_fit_time], 
              [lr_score, lr_time, lr_time, lr_time + lr_fit_time],
              [cbr_score, cbr_time, cbr_fit_time, cbr_time + cbr_fit_time],
              [cbr_no_ohe_score, cbr_no_ohe_time, cbr_no_ohe_fit_time, cbr_no_ohe_time + cbr_no_ohe_fit_time], 
              [lgmbr_score, lgmbr_time, lgmbr_fit_time, lgmbr_time + lgmbr_fit_time]]
pd.DataFrame(columns = columns, index = index, data = report)

,Значение RMSE,Время предсказания,Время обучения,Общее время
DummyRegressor,4822.094620,0.003934,0.003934,0.010476
LinearRegression,2782.322241,1.309621,1.309621,24.290045
CatBoostRegressor with OHE,1614.579969,0.183053,19.242517,19.425570
CatBoostRegressor without OHE,1640.609646,0.504584,44.740823,45.245407
LightGBMRegressor,1535.820785,6.116650,23.082368,29.199018


## Выводы

Лучшая метрика RMSE на валидационной выборке у модели - LGBMRegressor: 1535.820785. К тому же данная модель довольно быстро обучается. Но вот по времени предсказания - она самая медленная.

Второй результат у модели CatBoostRegressor на данных после OHE: 1614.579969. При этом она быстро обучается и еще быстрее предсказывает. 